In [1]:
import pandas as pd
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReadCSV").getOrCreate()

24/02/27 19:38:14 WARN Utils: Your hostname, Cameron-Wades-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.55 instead (on interface en0)
24/02/27 19:38:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/27 19:38:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load the dataset
df_spark = spark.read.csv("/Users/cameron-wadecarson/Desktop/CHALLENGES/project4/Resources/Carbon_Emission.csv", header=True, inferSchema=True)
df_spark.show()

+-----------+------+-----------+----------------+---------------------+------------+------------+---------------+--------------------+-----------------------------+---------------------------+--------------+----------------------+-------------------------+----------------------------+----------------------------+-----------------+--------------------+--------------------+--------------+
|  Body Type|   Sex|       Diet|How Often Shower|Heating Energy Source|   Transport|Vehicle Type|Social Activity|Monthly Grocery Bill|Frequency of Traveling by Air|Vehicle Monthly Distance Km|Waste Bag Size|Waste Bag Weekly Count|How Long TV PC Daily Hour|How Many New Clothes Monthly|How Long Internet Daily Hour|Energy efficiency|           Recycling|        Cooking_With|CarbonEmission|
+-----------+------+-----------+----------------+---------------------+------------+------------+---------------+--------------------+-----------------------------+---------------------------+--------------+-------------

In [3]:
df = df_spark.toPandas()
print(df.head())

    Body Type     Sex         Diet How Often Shower Heating Energy Source  \
0  overweight  female  pescatarian            daily                  coal   
1       obese  female   vegetarian  less frequently           natural gas   
2  overweight    male     omnivore  more frequently                  wood   
3  overweight    male     omnivore      twice a day                  wood   
4       obese  female   vegetarian            daily                  coal   

      Transport Vehicle Type Social Activity  Monthly Grocery Bill  \
0        public         None           often                   230   
1  walk/bicycle         None           often                   114   
2       private       petrol           never                   138   
3  walk/bicycle         None       sometimes                   157   
4       private       diesel           often                   266   

  Frequency of Traveling by Air  Vehicle Monthly Distance Km Waste Bag Size  \
0                    frequently      

In [7]:
# Assuming df is your DataFrame and the columns you listed are the ones with categorical data
categorical_columns = ['Body Type', 'Sex', 'Diet', 'How Often Shower', 'Heating Energy Source', 'Transport', 'Vehicle Type', 'Social Activity', 'Frequency of Traveling by Air', 'Waste Bag Size', 'Energy efficiency', 'Recycling', 'Cooking_With']

# Convert categorical variables into numerical ones
df2 = pd.get_dummies(df, columns=categorical_columns)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [9]:
# Split data into inputs and target
X = df2.drop(columns=['CarbonEmission'])
y = df2['CarbonEmission']

In [10]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Create a Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [12]:
# Train the model
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [26]:
# read in our personal data excel sheet
personal_data = pd.read_excel("personal_data.xlsx")
personal_data.head()

,Body Type,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,Recycling,Cooking_With,CarbonEmission
0,normal,male,omnivore,daily,electricity,private,petrol,often,250,rarely,689,medium,4,8,3,16,sometimes,['metal'],['stove'],NaN


In [27]:
personal_data_encoded = pd.get_dummies(personal_data)

# ensure all columns present in the training data are also present in personal data after one-hot encoding
missing_cols = set(X.columns) - set(personal_data_encoded.columns)
for col in missing_cols:
    personal_data_encoded[col] = 0

# reorder columns to match the order in the training data
personal_data_encoded = personal_data_encoded[X.columns]

predictions = model.predict(personal_data_encoded)

average_carbon_emission = df['CarbonEmission'].mean()

# compare predictions with average
for prediction in predictions:
    if prediction > average_carbon_emission:
        print("Your predicted carbon emissions are above the average.")
    else:
        print("Your predicted carbon emissions are below the average.")

Your predicted carbon emissions are below the average.
